# Groupby と Resample 

- 参照
    - [Group by: split-apply-combine — pandas 1.4.1 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#group-by-split-apply-combine)
    - [Resampling — pandas 1.4.1 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling)


## Groupbyとは

1. 1つのデータを複数のグループに分割する(Splitting)
1. 分割した各データに関数を適用して値を得る (Applying)
1. 得た値をデータにまとめる  (Combining)


In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    {
        "date": pd.date_range(start="2000-1-1 0:0:0", periods=9, freq="H"),
        "class": np.array(["A", "B", "C"]).repeat(3),
        "value A": np.random.random_sample(9),
        "value B": np.random.random_sample(9) * 100,
    }
)

In [ ]:
df

In [ ]:
# class 毎にデータを分割
df_grouped = df.groupby(by="class")


In [ ]:
# グループ化した各データの "value A" カラムに max 関数を適用
mx = df_grouped[["value A", "value B"]].max()


In [ ]:
# 得た値をデータにまとめる
mx

## Resample とは

- "時間"で Groupby すること
- 金融データは時系列データの事が多いので多用
- 注意：
    - Datetime index や Period index といった、時間を表す index を持つデータにのみ使える
    - "Frequency String" と呼ばれる文字列を使って、どの単位時間でデータを分割するかを設定する

In [ ]:
# df は、datetimeindex を持っていないので、
# date コラムを、このデータのインデックスに設定して上書き
df.set_index("date", inplace=True)
df 

In [ ]:
df.index

In [ ]:
# 2時間毎に、Value Aと Value Bの Max を取得
df.resample("2H")[["value A", "value B"]].max()

## 金融データのResampling

1. データを分割する(Splitting) 
    - 時系列データをとある時間単位に分割
1. 分割した各データに関数を適用して値を得る (Applying)  
    -  メソッド、apply、agg 
1. 得た値をデータにまとめる  (Combining) 
    - → 


In [ ]:
import asyncio

import nest_asyncio
import pandas as pd
import plotly.graph_objects as go
import pybotters
from IPython.display import HTML

nest_asyncio.apply()

## API から Trade データを取得

- 取引データから OHLCV を作成します
- [Get trades – API Documentation](https://docs.ftx.com/#get-trades)


In [ ]:
async def get_trades(market_name, start_time, end_time):
    async with pybotters.Client(
        apis={"ftx": ["", ""]}, base_url="https://ftx.com/api"
    ) as client:
        res = await client.get(
            f"/markets/{market_name}/trades",
            params={
                "start_time": start_time,
                "end_time": end_time,
            },
        )
        return await res.json()


data = asyncio.run(get_trades("BTC-PERP", 1643641200, 1643727600))
data["result"][:3]

## DataFrame の作成
- `time` コラムを datetimeindex に持つDataFrameを作成します

In [ ]:
df = pd.DataFrame(data["result"])
df.head()

In [ ]:
df.dtypes


In [ ]:
# time を datetime 型に変更し、この dataframe の index として設定
df = pd.DataFrame(data["result"])
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)
df.sort_index(inplace=True)
df

### 例：取引データから1分足と出来高を作成

In [ ]:
# まずは dataframe の resample メソッドを使って、Resampler object を作成
df_resampled = df.resample("1min", label="right")
df_ohlc = df_resampled["price"].ohlc()
df_ohlc["volume"] = df_resampled["size"].sum()
df_ohlc

### 詳しく説明

- TBA
